In [60]:
# TODO: add score of game as the target variable - should be all-strength on-ice goals for + off ice all strength goals for  - all-strength on-ice goals against - off ice all strength goals against

In [61]:
# notebook for scraping a player's career data from MoneyPuck.com
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [62]:
team_id_dict = {
    'NJD': 1,
    'PHI': 4,
    'LAK': 26,
    'TBL': 14,
    'BOS': 6,
    'NYR': 3,
    'PIT': 5,
    'DET': 17,
    'SJS': 28,
    'NSH': 18,
    'VAN': 23,
    'CHI': 16,
    'OTT': 9,
    'MTL': 8,
    'MIN': 30,
    'WSH': 15,
    'STL': 19,
    'ANA': 24,
    'PHX': 27,
    'NYI': 2,
    'TOR': 10,
    'FLA': 13,
    'BUF': 7,
    'CGY': 20,
    'COL': 21,
    'DAL': 25,
    'CBJ': 29,
    'WPG': 52,
    'EDM': 22,
    'VGK': 54,
    'CAR': 12,
    'ARI': 53,
    'ATL': 11
}

game_df = pd.read_csv('/Users/tylerviducic/dev/hockey_analytics/gamescore_model/kaggle_data/game.csv')
team_stats_df = pd.read_csv('/Users/tylerviducic/dev/hockey_analytics/gamescore_model/kaggle_data/game_teams_stats.csv')


In [63]:
# clean the player's data
# 8471675 = Sidney Crosby
# 8481559 = Jack Hughes

player_df = pd.read_csv('/Users/tylerviducic/dev/hockey_analytics/gamescore_model/mp_data/8471675.csv')
# print(player_df.to_markdown())
column_list = player_df.columns.to_list()
goal_col = [col for col in column_list if 'goal'in col or 'Goal' in col]
# print(goal_col)
max_len = max([len(x) for x in goal_col])
# print(max_len)
for i, col in enumerate(goal_col):
    end = '\n'
    if (i+1) % 4 != 0:
        end = '|'
    line = f' {col} ' + (' ' * (max_len - len(col)))
    # print(line, end=end)



In [64]:
bio_labels = ['playerId', 'season', 'name', 'gameId', 'playerTeam', 'opposingTeam', 'home_or_away', 'gameDate', 'position']
individual_skater_labels = ['icetime', 'shifts', 'gameScore', 'iceTimeRank', 'I_F_xOnGoal', 'I_F_xGoals', 'I_F_xRebounds', 'I_F_xFreeze', 'I_F_xPlayStopped', 'I_F_xPlayContinuedInZone', 'I_F_xPlayContinuedOutsideZone', 'I_F_flurryAdjustedxGoals', 'I_F_scoreVenueAdjustedxGoals', 'I_F_flurryScoreVenueAdjustedxGoals', 'I_F_primaryAssists', 'I_F_secondaryAssists', 'I_F_shotsOnGoal', 'I_F_missedShots', 'I_F_blockedShotAttempts', 'I_F_shotAttempts', 'I_F_points', 'I_F_goals', 'I_F_rebounds', 'I_F_reboundGoals', 'I_F_freeze', 'I_F_playStopped', 'I_F_playContinuedInZone', 'I_F_playContinuedOutsideZone', 'I_F_savedShotsOnGoal', 'I_F_savedUnblockedShotAttempts', 'penalties', 'I_F_penalityMinutes', 'I_F_faceOffsWon', 'I_F_hits', 'I_F_takeaways', 'I_F_giveaways', 'I_F_lowDangerShots', 'I_F_mediumDangerShots', 'I_F_highDangerShots', 'I_F_lowDangerxGoals', 'I_F_mediumDangerxGoals', 'I_F_highDangerxGoals', 'I_F_lowDangerGoals', 'I_F_mediumDangerGoals', 'I_F_highDangerGoals', 'I_F_scoreAdjustedShotsAttempts', 'I_F_unblockedShotAttempts', 'I_F_scoreAdjustedUnblockedShotAttempts', 'I_F_dZoneGiveaways', 'I_F_xGoalsFromxReboundsOfShots', 'I_F_xGoalsFromActualReboundsOfShots', 'I_F_reboundxGoals', 'I_F_xGoals_with_earned_rebounds', 'I_F_xGoals_with_earned_rebounds_scoreAdjusted', 'I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted', 'I_F_shifts', 'I_F_oZoneShiftStarts', 'I_F_dZoneShiftStarts', 'I_F_neutralZoneShiftStarts', 'I_F_flyShiftStarts', 'I_F_oZoneShiftEnds', 'I_F_dZoneShiftEnds', 'I_F_neutralZoneShiftEnds', 'I_F_flyShiftEnds', 'faceoffsWon', 'faceoffsLost', 'timeOnBench', 'penalityMinutes', 'penalityMinutesDrawn', 'penaltiesDrawn', 'shotsBlockedByPlayer']
on_ice_skater_labels = ['OnIce_F_xOnGoal', 'OnIce_F_xGoals', 'OnIce_F_flurryAdjustedxGoals', 'OnIce_F_scoreVenueAdjustedxGoals', 'OnIce_F_flurryScoreVenueAdjustedxGoals', 'OnIce_F_shotsOnGoal', 'OnIce_F_missedShots', 'OnIce_F_blockedShotAttempts', 'OnIce_F_shotAttempts', 'OnIce_F_goals', 'OnIce_F_rebounds', 'OnIce_F_reboundGoals', 'OnIce_F_lowDangerShots', 'OnIce_F_mediumDangerShots', 'OnIce_F_highDangerShots', 'OnIce_F_lowDangerxGoals', 'OnIce_F_mediumDangerxGoals', 'OnIce_F_highDangerxGoals', 'OnIce_F_lowDangerGoals', 'OnIce_F_mediumDangerGoals', 'OnIce_F_highDangerGoals', 'OnIce_F_scoreAdjustedShotsAttempts', 'OnIce_F_unblockedShotAttempts', 'OnIce_F_scoreAdjustedUnblockedShotAttempts', 'OnIce_F_xGoalsFromxReboundsOfShots', 'OnIce_F_xGoalsFromActualReboundsOfShots', 'OnIce_F_reboundxGoals', 'OnIce_F_xGoals_with_earned_rebounds', 'OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted', 'OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted', 'OnIce_A_xOnGoal', 'OnIce_A_xGoals', 'OnIce_A_flurryAdjustedxGoals', 'OnIce_A_scoreVenueAdjustedxGoals', 'OnIce_A_flurryScoreVenueAdjustedxGoals', 'OnIce_A_shotsOnGoal', 'OnIce_A_missedShots', 'OnIce_A_blockedShotAttempts', 'OnIce_A_shotAttempts', 'OnIce_A_goals', 'OnIce_A_rebounds', 'OnIce_A_reboundGoals', 'OnIce_A_lowDangerShots', 'OnIce_A_mediumDangerShots', 'OnIce_A_highDangerShots', 'OnIce_A_lowDangerxGoals', 'OnIce_A_mediumDangerxGoals', 'OnIce_A_highDangerxGoals', 'OnIce_A_lowDangerGoals', 'OnIce_A_mediumDangerGoals', 'OnIce_A_highDangerGoals', 'OnIce_A_scoreAdjustedShotsAttempts', 'OnIce_A_unblockedShotAttempts', 'OnIce_A_scoreAdjustedUnblockedShotAttempts', 'OnIce_A_xGoalsFromxReboundsOfShots', 'OnIce_A_xGoalsFromActualReboundsOfShots', 'OnIce_A_reboundxGoals', 'OnIce_A_xGoals_with_earned_rebounds', 'OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted', 'OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted']
all_strength_labels = bio_labels + individual_skater_labels

In [65]:
grouped_df = player_df.groupby('gameId')

In [66]:
# TODO: make this player team agnostic
def get_score_of_game(game_id, team, team_id_dict=None, game_df=None, team_stats_df=None):
    """
    returns the score of a game id if it ended in regulation
    return order is home score, away score
    """
    if game_df is None:
        game_df = pd.read_csv('/Users/tylerviducic/dev/hockey_analytics/gamescore_model/kaggle_data/game.csv')
    if team_stats_df is None:
        team_stats_df = pd.read_csv('/Users/tylerviducic/dev/hockey_analytics/gamescore_model/kaggle_data/game_teams_stats.csv')
    if team_id_dict is None:
        team_id_dict = {
        'NJD': 1,
        'PHI': 4,
        'LAK': 26,
        'TBL': 14,
        'BOS': 6,
        'NYR': 3,
        'PIT': 5,
        'DET': 17,
        'SJS': 28,
        'NSH': 18,
        'VAN': 23,
        'CHI': 16,
        'OTT': 9,
        'MTL': 8,
        'MIN': 30,
        'WSH': 15,
        'STL': 19,
        'ANA': 24,
        'PHX': 27,
        'NYI': 2,
        'TOR': 10,
        'FLA': 13,
        'BUF': 7,
        'CGY': 20,
        'COL': 21,
        'DAL': 25,
        'CBJ': 29,
        'WPG': 52,
        'EDM': 22,
        'VGK': 54,
        'CAR': 12,
        'ARI': 53,
        'ATL': 11
        }
    team_id = team_id_dict[team]
    game = game_df.loc[game_df.game_id == game_id]
    if len(game.outcome) == 0:
        print(f'gameId with no outcome: {game_id}')
        return None, None
    outcome = game.outcome.values[0].split(' ')[-1]
    if outcome == 'OT':        
        game = team_stats_df.loc[team_stats_df.game_id == game_id]
        players_team = game.loc[game.team_id == team_id]
        opponent_team = game.loc[game.team_id != team_id]
        player_team_score = players_team.goals.values[0]
        opponent_team_score = opponent_team.goals.values[0]
        if players_team.HoA.values[0] == 'away':
            return opponent_team_score, player_team_score
        else:
            return player_team_score, opponent_team_score
    else:
        home_score = game.home_goals.values[0]
        away_score = game.away_goals.values[0]
        return home_score, away_score


def reduce_df(grouped_df):
    all_strength_df = pd.concat([group.loc[group['situation'] == 'all', all_strength_labels].reset_index(drop=True) for game, group in grouped_df])
    oi_5v5_frames_df = pd.concat([group.loc[group['situation'] == '5on5', on_ice_skater_labels].reset_index(drop=True) for game, group in grouped_df])
    reduced_df = pd.concat([all_strength_df, oi_5v5_frames_df], axis=1)
    reduced_df[['home_score', 'away_score']] = reduced_df.apply(lambda row: get_score_of_game(row['gameId'], row['playerTeam'], team_id_dict=team_id_dict, game_df=game_df, team_stats_df=team_stats_df), axis=1)
    return 

In [67]:
# def reduce_df(grouped_df):
#     all_strength_df = pd.concat([group.loc[group['situation'] == 'all', all_strength_labels].reset_index(drop=True) for game, group in grouped_df])
#     oi_5v5_frames_df = pd.concat([group.loc[group['situation'] == '5on5', individual_skater_labels].reset_index(drop=True) for game, group in grouped_df])
#     game_team_df = player_df.groupby(['gameId', 'playerTeam']).size().reset_index(name='counts')
#     all_strength_df = pd.merge(all_strength_df, game_team_df, on=['gameId'], how='left')
#     reduced_df = pd.concat([all_strength_df, oi_5v5_frames_df], axis=1)
#     reduced_df[['home_score', 'away_score']] = reduced_df.apply(lambda row: get_score_of_game(row['gameId'], row['playerTeam']), axis=1, result_type='expand')
#     return reduced_df

reduced_df = reduce_df(grouped_df)
print(reduced_df.head().to_markdown())

gameId with no outcome: 2020020001
gameId with no outcome: 2020020018
gameId with no outcome: 2020020035
gameId with no outcome: 2020020050
gameId with no outcome: 2020020068
gameId with no outcome: 2020020083
gameId with no outcome: 2020020093
gameId with no outcome: 2020020109
gameId with no outcome: 2020020125
gameId with no outcome: 2020020144
gameId with no outcome: 2020020151
gameId with no outcome: 2020020166
gameId with no outcome: 2020020181
gameId with no outcome: 2020020201
gameId with no outcome: 2020020215
gameId with no outcome: 2020020242
gameId with no outcome: 2020020257
gameId with no outcome: 2020020267
gameId with no outcome: 2020020281
gameId with no outcome: 2020020302
gameId with no outcome: 2020020313
gameId with no outcome: 2020020328
gameId with no outcome: 2020020341
gameId with no outcome: 2020020366
gameId with no outcome: 2020020393
gameId with no outcome: 2020020405
gameId with no outcome: 2020020416
gameId with no outcome: 2020020431
gameId with no outco

ValueError: Columns must be same length as key